# Objetivo de este notebook

Con este notebook se exhibiran algunos conceptos, algoritmos o funciones para establecer un modelo de recomendación de películas

# Requisitos

In [12]:
#Para correr las funciones de la librería libmf de python de este notebook
#se debe clonar el repo de https://github.com/PorkShoulderHolder/python-libmf.git
#y subir este notebook dentro de la carpeta /libmf/tests/

#Para correr las funciones de libmf de este notebook ejecutados con 
#Shell Commands ("!") y Shell-Related Magic Commands ("%")
#se debe clonar el repo de https://github.com/cjlin1/libmf.git

#El clonar estos archivos es necesario porque contienen funciones programadas o
#ejemplos construidos para exhibir el uso de estos paquetes

#Existen bases que se requieren tener en la carpeta donde se corra el código
#a saber: movies_title_fix.csv dat_muestra_nflix.csv

#Si se requiere correr este notebook a través de docker con la imagen de
#jupyer numerical, se deben instalar los paquetes libmf y colorama,
#ejecutando los comandos de abajo, reiniciando el kernel (luego
#se puede comentar con "#" los "pip install" para que no se vuelvan a ejecutar)

In [13]:
#pip install libmf

In [14]:
#pip install colorama

In [1]:
import sys
import numpy as np
import types
import scipy
from scipy.sparse import random
from scipy import stats
import pandas as pd
import copy
import random
from functools import partial
from colorama import init

import libmf
from libmf import mf

Using file found in /home/miuser/.local/lib/python3.6/site-packages:
/home/miuser/.local/lib/python3.6/site-packages/libmf.cpython-36m-x86_64-linux-gnu.so


# Funciones auxiliares
Para explicar y ejemplificar el uso de libmf

In [2]:
#Función auxiliar para generar matrices ralas (sólo se usa para un ejemplo)
def constante(x,useless):
    """
    Devuelve la constante x
    
    params: x       float constante que se quiere devolver
            useless float cualquier valor que se quiera pasar
    return: x       Devuelve la misma constante
    """
    return x

In [3]:
#Función auxiliar para generar matrices ralas (sólo se usa para un ejemplo)
def perform(fun, *args):
    """
    Evalúa una función con los argumentos indicados
    
    params: fun         Una función
            *args       Parámetros que utiliza la función "fun"
                        (se pueden meter cuantos parámetros tenga 
                        dicha funcion)
    return: fun(*args)  Valor de fun con parámetros *args
    """
    return fun(*args)

In [4]:
#Función auxiliar para generar matrices ralas (sólo se usa para un ejemplo)
def nan_or_value(x, f, *args):
    """
    Devuelve el valor de una función evaluada con sus parámetros
    *args acorde a la variable indicadora x
    
    params: x         boolean, si es 1 tómese valor de la función
                      si no será nan o 0 según especifique nan_zero
            fun       Una función
            *args     Parámetros que utiliza la función "fun"
                      (se pueden meter cuantos parámetros tenga 
                      dicha funcion)
            nan_zero  Si no se especifica nan devuelve 0
    return: fun(*args)  Valor de fun con parámetros *args
    """
    if(x==1):
        return perform(f,*args)
    else:
        return np.nan

In [5]:
#Función auxiliar para generar matrices ralas (sólo se usa para un ejemplo)

#Con esta vectorización se podrá aplicar a todos los elementos
#de un np.array la función nan_or_value
nan_or_value_vec=np.vectorize(nan_or_value, otypes=[object])

In [6]:
#Función para generar matrices ralas (sólo se usa para un ejemplo)
#Por su forma de construcción, esta función es bastante ineficiente para
#ejemplos grandes
def crear_matriz_rala(n_ren,m_col,prop_elem_difer_nan,func_rand,*args_func_rand):
    """
    Construye matriz rala de n_renxm_col, seleccionando al azar los elementos
    que serán nan cumpliendo que la proporción de éstos es de 
    1-prop_elem_difer_nan, donde los numeros diferentes de nan serán aleatorios
    generados a partir de una funcion aletorea que se envíe como parámetro
    (y sus argumentos)
    
    params: n_ren               int numero de renglones que tiene la matriz
            m_col               int numero de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
            func_rand           Función que genera un sólo no. aleatorio de
                                tipo float (no arreglos)
            *args_fun_rand      Argumentos necesarios para ejecutar la
                                la funcoión fun_rand
            nan_zero            Si no se especifica nan devuelve 0
    return: nan_or_value_vec    np.ndarray que contiene matriz rala
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    #se juntan 2 arreglos, uno de ceros y otro de unos, para después
    #reacomodarlos al azar
    M=np.concatenate((np.zeros(tot-k).astype(int), np.ones(k).astype(int)))
    M=np.array(random.sample(list(M),tot))
    M=M.reshape(n_ren,m_col)
    #Los unos se intercambias por numeros aleatorios y los ceros
    #por nan (o se dejan como cero)
    M=nan_or_value_vec(M,func_rand,*args_func_rand).astype(float)
    return M

In [7]:
#Función auxiliar para generar matrices ralas (sólo se usa para un ejemplo)
def compactar_matriz_rala(M):
    """
    Pasa una matriz rala a su expresión compacta
    
    params: M  np.ndarray que representa matriz rala
    return: MC np.ndarray matriz en expresión compacta
               (no se reportarán los elementos con valor
               nan)
    """
    if type(M) is not np.ndarray:
        sys.exit('M debe ser de tipo numpy.ndarray')
    elif np.isnan(M).sum()==len(M):
        sys.exit('Todos los elementos de M son nan')
    MC=np.zeros((M[np.isnan(M)==False].size,3))
    k = 0
    n,m=M.shape
    for i in range(n): 
        for j in range(m): 
            if (~np.isnan(M[i,j])): 
                MC[k,0] = i
                MC[k,1] = j
                MC[k,2] = M[i,j] 
                k += 1
    return MC

In [8]:
#Función para generar matrices ralas en su expresión compacta
#Utilizada para varios ejemplos
def crear_matriz_rala_compacta(n_ren, m_col, prop_elem_difer_nan):
    """
    Construye matriz rala de n_renxm_col, seleccionando al azar los elementos
    que serán distintos de nan. Éstos elementos distintos de nan serán
    numero aleatorios entre 1 y 5 (uniforme discreta). Se dice compacta porque
    es una matriz solo señala las coordenadas o entredas de la matriz que 
    son diferentes a nan
    
    params: n_ren               int numero de renglones que tiene la matriz
            m_col               int numero de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
    return: M  np.ndarray que contiene matriz rala "compacta"
               (1era columna representa el índice de renglones,
               2da columna representa el índice de columnas,
               3era columna representa el valor del elemento
               o calificación en el contexto de netflix)
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    M=random.sample(range(tot), k)
    columnas=np.mod(M,m_col)
    renglones=np.divide(M,m_col).astype(int)
    aleatorios=np.random.randint(1, 6, k)
    M=np.concatenate([renglones, columnas, aleatorios])
    M=M.reshape(3,k).T
    return M

In [9]:
#Función auxiliar al procesamiento de datos para la aplicación de un
#modelo de recomendación
def modelo_base_ref(MC):
    """
    Obtiene promedios por índices de renglón y columnas, así como del total
    (o calificaciones en el contexto de películas netflix)
    
    params: MC np.ndarray (con 3 columnasm, la 1era se toma como la
               de índices de renglon, la 2da como la de índices de columna
               y la 3era como el valor de los elementos o calificaciones en
               el contexto de netflix)
    return: count_by  pd.DataFrame de 3 columnas que tiene sólo los índices de
                      renglones o columnas que tienen al menos n elementos
    """
    df=pd.DataFrame(MC,columns=['row','col','score'])
    #Media total
    c=df['score'].mean(axis=0)

    #media por columna
    mean_by_col=df[['col','score']].groupby(['col']).mean()
    mean_by_col.rename(columns={'score':'mean_score_by_col'}, inplace=True)
    #media por renglón
    mean_by_row=df[['row','score']].groupby(['row']).mean()
    mean_by_row.rename(columns={'score':'mean_score_by_row'}, inplace=True)

    #se unen todas los resultados obtenidos, incluyendo la columna de
    #índice de columnas y renglones
    df=df.set_index('col').join(mean_by_col,how='left')
    df['col']=df.index
    df=df.set_index('row').join(mean_by_row,how='left')
    df['row']=df.index
    df=df[['row','col','mean_score_by_row','mean_score_by_col']]
    df['total_score_mean']=c
    
    return df

In [10]:
#Función auxiliar al procesamiento de datos para la aplicación de un
#modelo de recomendación
def filtrar(MC,ren_o_col,n):
    """
    Filtra los índices de renglon o columna que tengan más de n elementos
    (o calificaciones en el contexto de películas netflix)
    
    params: MC        np.ndarray (con 3 columnas, de nombre indistinto)
            ren_o_col string igual a 'row' o 'col'
            n         int no. elementos mínimos requeridos
    
    return: count_by  np.ndarray de 3 columnas que tiene sólo los índices de
                      renglones o columnas que tienen al menos n elementos
    """
    MC_df=pd.DataFrame(MC,columns=['row','col','score'])
    
    #este pequeño bloque calcula el total de elementos por columna o renglón
    count_by=MC_df[[ren_o_col,'score']].groupby([ren_o_col]).count()
    count_by.rename(columns={'score':'tot_counted'}, inplace=True)
    count_by[ren_o_col]=count_by.index
    count_by=count_by[[ren_o_col,'tot_counted']]
    
    #en este bloque se filtran los elementos de la matriz cuyo conteo por
    #columna o renglón tienen más de n elementos
    count_by=count_by[count_by.apply(lambda x: x['tot_counted'] >= n, axis=1)]
    count_by=MC_df.set_index(ren_o_col).join(count_by.set_index(ren_o_col),how='inner')
    count_by[ren_o_col]=count_by.index
    count_by=count_by[['row','col','score']]
    
    return count_by.to_numpy()

In [11]:
#Función auxiliar para obtención de muestras aletorias
def muestra_val_ent(df,prop):
    """
    Separa al azar una base dada en 2 bases acorde una proporción de elemtos
    dada
    
    params: df    pd.DataFrame (con 3 columnas=['usuario_id','peli_id',
                  'calif_x'])
            prop  float (0 < prop < 1) proporción de id_usuarios y
                  proporción de id_peliculas que se seleccionarán
    
    return: data_test      pd.DataFrame que tiene base que se usará
                           para prueba (por construcción el numero de
                           elementos de esta base no necesariamente coincide
                           con prop*total de elementos de df)
            data_training  pd.DataFrame que tiene base que se usará
                           para entrenamiento
    """
    #Base de todos los usuarios
    usuarios=pd.DataFrame(df['usuario_id'].unique(), columns=['usuario_id'])
    #Muestreo de usuarios
    valida_usuarios=usuarios.sample(frac=prop)  

    #Base de todas las películas
    peliculas=pd.DataFrame(df['peli_id'].unique(), columns=['peli_id'])
    #Muestreo de películas
    valida_pelis=peliculas.sample(frac=prop)
    
    #df_valida tiene las columnas usuario_id, peli_id y calif, pero
    #sólo incluye los usuarios muestreados en valida_usuarios
    df_valida=pd.merge(df,valida_usuarios, on ='usuario_id', how='inner')
    #data_valida filtra de la base df_valida los pelis_id muestreas
    #en valida_pelis
    data_test=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
    left=pd.merge(df, data_test, how='left',on=['usuario_id', 'peli_id'])
    data_training=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]
    return data_test, data_training

In [12]:
def imprime_atributos_MF(Obj):
    """
    Imprime los parámetros utilizados para factorización de la matriz contenida
    en Obj, mismos que son atributos del atributo _options
    
    params: Obj  libmf.mf.MF objeto de la librería libmf-python que contiene
                 la información requerida para hacer la factorización deseada
    
    return: None No regresa nada, sólo imprime los parámetros utilizados para
                 hacer la factorización
    """
    if type(Obj) is not libmf.mf.MF:
        sys.exit('A y b deben ser de tipo libmf.mf.MF')
    print('Atributos del modelo:')
    print('fun={0:d}'.format(Obj._options.fun))
    print('k={0:d}'.format(Obj._options.k))
    print('nr_threads={0:d}'.format(Obj._options.nr_threads))
    print('nr_bins={0:d}'.format(Obj._options.nr_bins))
    print('nr_iters={0:d}'.format(Obj._options.nr_iters))
    print('lambda_p1={0:0.4f}'.format(Obj._options.lambda_p1))
    print('lambda_p2={0:0.4f}'.format(Obj._options.lambda_p2))
    print('lambda_q1={0:0.4f}'.format(Obj._options.lambda_q1))
    print('lambda_q2={0:0.4f}'.format(Obj._options.lambda_q2))
    print('eta={0:0.4f}'.format(Obj._options.eta))
    print('do_nmf=',Obj._options.do_nmf)
    print('quiet=',Obj._options.quiet)
    print('copy_data=',Obj._options.copy_data)
    return None

In [13]:
def imprime_atributos_model(Obj):
    """
    Imprime los atributos del objeto model (definido dentro dentro de la
    librería libmf)
    
    params: Obj  libmf.mf.MFModel objeto de la librería libmf-python que
                 contiene los resultados obtenidos de la corrida del
                 modelo de factorización
                 
    return: None No regresa nada, sólo imprime los resultados contenidos
            en el objeto model, generado con los métodos del objeto MF
    """
    if type(Obj) is not libmf.mf.MFModel:
        sys.exit('A y b deben ser de tipo libmf.mf.MF')
    print('Resultados model:')
    print('fun={0:d}'.format(Obj.fun))
    print('m={0:d}'.format(Obj.m))
    print('n={0:d}'.format(Obj.n))
    print('k={0:d}'.format(Obj.m))
    print('b={0:0.4f}'.format(Obj.b))
    print('P={0:0.4f}'.format(Obj.P.contents.value))
    print('Q={0:0.4f}'.format(Obj.Q.contents.value))
    return None

In [14]:
#Elegiremos un formato de general para visualizar las matrices exhibidas como ejemplo
np.set_printoptions(sign=' ', precision=3)
#np.set_printoptions(sign=' ',formatter={'float': lambda x: "{0:0.3f}".format(x)})

# Matrices ralas (sparse)

In [18]:
#En este proyecto se exhibe el uso de las funciones de la librería libmf
#relacionadas a la factorización de matrices. El proyecto se enfoca al uso
#de estos algoritmos/métodos para los sistemas de recomendación de películas
#cuyo objetivo será obtener recomendaciones de películas para usuarios
#contenidos dentro de una base de datos proporcionada por Netflix

#Si bien la base tiene datos como peli_id, usuario_id, calif y fecha, el
#algoritmo solo trabajará con los datos de calificaciones. Si bien existen
#diversos usuarios y películas, son pocas películas de las que se tienen
#calificaciones (escala del 1 al 5) proporcionadas por usuarios, por lo que
#la representación de estas calificaciones a través de una matriz (donde
#los usuarios serán los índices de los renglones, las películas los índices
#de columnas y las calificaciones los valores de las entradas de la matriz),
#contendrá muchos huecos o calificaciones inexistentes,
#es decir, es una matriz rala (llena de ceros o valores no proporcionados)
#No obstante lo anterior, debido al numero de usuarios y películas
#existen muchos datos, y para no guardar todos los valores de cero (o NA)
#se opta por expresar y guardar dicha matriz en una forma compacta,
#la cual contiene por cada calificación el usuario que la generó y la película
#a la que corrresponde (es decir, es una matriz con 3 columnas)

In [15]:
#Crear matriz rala de tamaño 20x5, donde el 25% de sus elementos
#será distinto de nan, señalando la distribución de probabilidad
#que tendrán dichos elementos
M=crear_matriz_rala(20,5,0.25,stats.randint.rvs,*[1,6])
#Se puede cambiar la distribución aleatoria, probar con las
#distribuciones existentes en scipy.stats o la función constante
#R=crear_matriz_rala(20,5,0.5,scipy.stats.uniform.rvs,*[1,4])
#R=crear_matriz_rala(20,5,0.2,stats.binom.rvs,*[5,0.2])
#R=crear_matriz_rala(20,5,0.5,constante,*[1,0])
print('Matriz M:\n',M)
print('Tamaño M:\n',M.shape)
print('Numero de elementos diferentes de nan:\n',M[np.isnan(M)==False].size)

Matriz M:
 [[  1.  nan  nan   3.   4.]
 [  3.  nan  nan  nan   3.]
 [  4.  nan   2.   2.   5.]
 [ nan  nan  nan  nan  nan]
 [ nan  nan   4.  nan   3.]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan   5.  nan  nan]
 [  2.  nan  nan   2.  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan   3.]
 [ nan  nan  nan  nan  nan]
 [ nan  nan   2.  nan   4.]
 [  4.  nan  nan   4.  nan]
 [  1.  nan   2.  nan  nan]
 [  3.  nan   5.  nan  nan]
 [ nan  nan   1.   3.  nan]
 [ nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan]]
Tamaño M:
 (20, 5)
Numero de elementos diferentes de nan:
 25


In [16]:
#Expresión compacta de la matriz rala (con 3 columnas: col1,col2,col3),
#que señala el índice de renglón (col1) y columna (col2) de la matriz R,
#correspondiente a los elementos distitnos de nan, así como el valor
#de dicho elemento (col3)
MC=compactar_matriz_rala(M)
print('Expresión compacta de M (MC):\n',MC)
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC):
 [[  0.   0.   1.]
 [  0.   3.   3.]
 [  0.   4.   4.]
 [  1.   0.   3.]
 [  1.   4.   3.]
 [  2.   0.   4.]
 [  2.   2.   2.]
 [  2.   3.   2.]
 [  2.   4.   5.]
 [  4.   2.   4.]
 [  4.   4.   3.]
 [  8.   2.   5.]
 [  9.   0.   2.]
 [  9.   3.   2.]
 [ 11.   4.   3.]
 [ 13.   2.   2.]
 [ 13.   4.   4.]
 [ 14.   0.   4.]
 [ 14.   3.   4.]
 [ 15.   0.   1.]
 [ 15.   2.   2.]
 [ 16.   0.   3.]
 [ 16.   2.   5.]
 [ 17.   2.   1.]
 [ 17.   3.   3.]]
Tamaño MC:
 (25, 3)


In [17]:
#El presente trabajo plantea trabajar con bases de películas donde
#los ínidices i de renglones representan usuarios, los j de columnas
#las películas, y el valor de elemento de la matriz la calificación
#asignada a la película j por el usuario i
#Por lo anterior, para crear matrices compactas ralas usaremos funciones propias
#donde un usuario no califique más de una vez una misma película (lo cual 
#sucediería si se genera la matriz de calificaciones con la función de la
#paquetería libmf). La calificación será un no. aleatorio entre 1 y 5
#generada con una dist. uniforme discreta
MC=crear_matriz_rala_compacta(20,5,0.25)
print('Expresión compacta de M (MC) ordenada por renglon:\n',MC[np.argsort(MC[:,0])])
print('Tamaño MC:\n',MC.shape)

Expresión compacta de M (MC) ordenada por renglon:
 [[ 1  3  4]
 [ 2  4  1]
 [ 3  2  4]
 [ 4  4  1]
 [ 4  1  4]
 [ 5  2  1]
 [ 5  1  1]
 [ 7  2  1]
 [ 7  1  1]
 [ 9  2  5]
 [10  3  3]
 [10  4  1]
 [10  0  3]
 [11  2  5]
 [11  3  3]
 [11  1  2]
 [12  4  5]
 [12  3  5]
 [13  1  5]
 [14  2  5]
 [15  0  1]
 [16  1  5]
 [18  0  3]
 [18  1  2]
 [19  3  2]]
Tamaño MC:
 (25, 3)


In [18]:
#Aquí una forma de guardar la matriz compacta en un txt
#que podría ser utilizado directamente con las funciones
#de libmf a través de Shell Commands ("!")
np.savetxt('MC.txt', MC, delimiter=' ', fmt="%.0f")
#Se imprime lo contenido en el .txt
f = open('MC.txt', 'r')
file_contents = f.read()
print (file_contents)
f.close()

16 1 5
5 1 1
18 1 2
14 2 5
18 0 3
10 3 3
12 4 5
19 3 2
4 1 4
13 1 5
12 3 5
7 1 1
11 1 2
10 0 3
2 4 1
9 2 5
3 2 4
1 3 4
11 3 3
4 4 1
7 2 1
15 0 1
11 2 5
10 4 1
5 2 1



# Procesos útiles para construcción de modelo
### Calificaciones promedio por usuario y/o película, y filtrado

In [19]:
#Para plantear un "modelo base de referencia", se calculan las medias
#por renglones (mean_ren) y por columnas (mean_col), así como 
#una media total (mean_tot), luego se sustituyen los valores de 
#la matriz (x_ij) por: mean_ren + mean_col - mean_tot
#La función modelo_base_ref regresa los promedios por renglón,
#columna y total. Con estos promedios se puede construir un modelo
#para heterogenizar el uso de la escala (calificaciones)

promedios=modelo_base_ref(MC)
display(promedios.head())
#podemos convertir a numpy array para realizar operaciones con
#variables de este tipo

#Recuerda pasar a np.array para hacer operaciones
promedios=promedios.to_numpy()
#el modelo base de ref quedaría de la siguiente forma:
print('\n El modelo base de referencia es mean_ren + mean_col - mean_tot:') 
promedios[:,0]+promedios[:,1]-promedios[:,2]

,row,col,mean_score_by_row,mean_score_by_col,total_score_mean
row,,,,,
1,1,3,4.0,3.400000,2.92
2,2,4,1.0,2.000000,2.92
3,3,2,4.0,3.500000,2.92
4,4,1,2.5,2.857143,2.92
4,4,4,2.5,2.000000,2.92



 El modelo base de referencia es mean_ren + mean_col - mean_tot:


array([  0.   ,   5.   ,   1.   ,   2.5  ,   5.5  ,   5.   ,   6.   ,
         7.   ,   8.   ,   6.   ,   7.667,  10.667,  11.667,   8.667,
         9.667,  10.667,  10.   ,  11.   ,   9.   ,  11.   ,  14.   ,
        12.   ,  15.5  ,  16.5  ,  20.   ])

In [20]:
#En el camino para construir modelos adecuados, puede ser de interés
#o utilidad identificar los usuarios que tiene más películas calificadas
#o las películas con más calificaciones, más precisamente, que tengan
#más de "n" calificaciones.

#Usuarios que calificaron más de 2 películas
display(filtrar(MC,'row',2))
#Películas que fueron calificadas por más de 2 usuarios
display(filtrar(MC,'col',2))

array([[ 4,  1,  4],
       [ 4,  4,  1],
       [ 5,  1,  1],
       [ 5,  2,  1],
       [ 7,  1,  1],
       [ 7,  2,  1],
       [10,  3,  3],
       [10,  0,  3],
       [10,  4,  1],
       [11,  1,  2],
       [11,  3,  3],
       [11,  2,  5],
       [12,  4,  5],
       [12,  3,  5],
       [18,  1,  2],
       [18,  0,  3]])

array([[18,  0,  3],
       [10,  0,  3],
       [15,  0,  1],
       [16,  1,  5],
       [ 5,  1,  1],
       [18,  1,  2],
       [ 4,  1,  4],
       [13,  1,  5],
       [ 7,  1,  1],
       [11,  1,  2],
       [14,  2,  5],
       [ 9,  2,  5],
       [ 3,  2,  4],
       [ 7,  2,  1],
       [11,  2,  5],
       [ 5,  2,  1],
       [10,  3,  3],
       [19,  3,  2],
       [12,  3,  5],
       [ 1,  3,  4],
       [11,  3,  3],
       [12,  4,  5],
       [ 2,  4,  1],
       [ 4,  4,  1],
       [10,  4,  1]])

In [327]:
#AQUÍ HAY QUE PONER LO QUE PUSO JAVI RESPECTO A FILTRAR LAS PELIS Y USUARIOS QUE APARECEN MUCHO

# Librería libmf python

In [35]:
# Python-libmf contiene de fondo el mismo código que el de la librería libmf, 
# aunque tiene programadas clases con atributos y métodos para poder ejecutar 
# los métodos de libmf. A continuación se señalan las principales 
# clases, atributos y métodos

# Clase: MF
# Atributos: .model
#            ._options 
#            .i
#            .j
# Métodos: .fit -> < Objetivo: factorize the i x j data matrix X into (j, k) (k, i) 
#                  <           sized matrices stored in MF.model
#                  < Parámetro X: (n, 3) shaped numpy array [known index and values 
#                  <              of the data matrix]
#
#          .predict -> < Objetivo: assuming we have already run the fit method, predict
#                      <           the values at certain indices of the data matrix
#                      < Parámetro X: (n, 2) shaped numpy array
#                      < Regresa: numpy array of length n
#
#          .mf_cross_validation -> < Objetivo: Realizar cross-validation
#                                  < Parámetro X: (n, 3)
#                                  < Parámetro folds: number of train / test splits
#                                  < Regresa: score across all folds 
#
#          .q_factors -> < Objetivo: Obtener la matriz Q
#
#          .p_factors -> < Objetivo:  Obtener la matriz P


# Clase: model
# Atributos: .fun
#            .m 
#            .n
#            .k
#            .b
#            .P
#            .Q
# Métodos: NA


# Clase: _options
# Atributos: .fun
#            .k
#            .nr_threads
#            .nr_bins
#            .nr_iters
#            .lambda_p1
#            .lambda_p2
#            .lambda_q1
#            .lambda_q2
#            .eta
#            .do_nmf
#            .quiet
#            .copy_data
# Métodos: NA

In [36]:
#A continuación un comparativo de los parámetros usados por python-libmf 
#y libmf, así como sus valores predeterminados

|Parámetro:|Nombre python-libmf|Valor por default python-libmf|Nombre libmf|Valor por default libmf|
|:--:|:--:|:--:|:--:|:--:|
|Función de pérdida|fun|0|-f|0|
|Numero de factores latentes|k|8|-k|8|
|Numero de threads|nr_threads|12|-s|12|
|Numero de bins|nr_bins|26|-n|adjusted by LIBMF for speed|
|Numero de iteraciones|nr_iters|20|-t|20|
|Parámetro de regularización L1 para P y Q<sup>(1)</sup>|lambda_p1|0.04|-l1|0|
|Parámetro de regularización L2 para P y Q<sup>(1)</sup>|lambda_p2|0|-l2|0.1|
|Parámetro de regularización L1 para P y Q<sup>(1)</sup>|lambda_q1|0.04|-l1|0|
|Parámetro de regularización L2 para P y Q<sup>(1)</sup>|lambda_q2|0|-l1|0.1|
|Tasa inicial de aprendizaje|eta|0.1|-r|0.1|
|Realizar factorización matricial no negativa<sup>(3)</sup>|do_nmf|False|--nmf|False|
|Sin outputs<sup>(3)</sup>|quite|False|--quite|False|
|Copiar datos|copy_data|True|NA|NA|
|Ruta conjunto validación|NA|NA|-p|ND|
|Numero de folds para validación cruzada<sup>(4)</sup>|ND|5|-v|1|
|Coeficiente de pérdida de entradas negativas|NA|NA|-a|1|
|Valor de entradas negativas<sup>(2)</sup>|NA|NA|-c|0.0001|
|Realizar entrenamiento a nivel de disco<sup>(3)</sup>|NA|NA|--disk|False|

<sup>(1)</sup> En libmf "l1" y "l2" se pueden definir con 1 o 2 parámetros, si se define uno se entiende que aplica tanto para P como Q, si se definen 2 el 1ero aplica para P y el 2do para Q

<sup>(2)</sup> Se supone que cada entrada positiva es 1

<sup>(3)</sup> En libmf su ausencia indica que no se llevará acabo. Por ejemplo, si no aparece "--nmf" en el llamado de mf-train, implica que las matrices P y Q pueden tener elementos negativos

<sup>(3)</sup> Para python-libmf si bien no es un parámetro que se defina cuando se crea el objeto MF como el resto de parámetros, se puede ejecutar una función de validación cruzada

In [21]:
# A continuación se corre un DEMO que muestra un poco el uso de estas funciones
# resalta que para programar este demo se definieron unas clases que a su vez
# ocupan las clases principales antes mencionadas de python-limbf
#ASEGURESE DE PONER LA UBICACIÓN CORRECTA DE LA UBICACIÓN DE LA CARPETA python-libmf

#Esto sólo corre sí te ubicas en la carpeta que contien mf_tests.py
# %cd /home/miuser/python-libmf/tests
%cd /home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/python-libmf/tests
!pwd
!python3 mf_tests.py

/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/python-libmf/tests
/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/python-libmf/tests
/home/miuser/.local/lib/python3.6/site-packages/libmf/__init__.py
Using file found in /home/miuser/.local/lib/python3.6/site-packages:
/home/miuser/.local/lib/python3.6/site-packages/libmf.cpython-36m-x86_64-linux-gnu.so
iter      tr_rmse          obj
   0       0.5228   3.3969e+02
   1       0.4879   3.1376e+02
   2       0.4347   2.7306e+02
   3       0.3795   2.3204e+02
   4       0.3384   2.0252e+02
   5       0.3133   1.8385e+02
   6       0.2984   1.7168e+02
   7       0.2900   1.6339e+02
   8       0.2857   1.5780e+02
   9       0.2836   1.5401e+02
  10       0.2807   1.4982e+02
  11       0.2801   1.4747e+02
  12       0.2800   1.4567e+02
  13       0.2791   1.4374e+02
  14       0.2788   1.4245e+02
  15       0.2785   1.4122e+02
  16       0.2781   1.4004e+02
  17       0.2782   1.3926e+02
  18       0.2784   1.3866e+

In [22]:
#se tiene una función para generar matrices ralas compactas sin embargo
#se observa que puede arrojar más de un valor para un mismo
#elemento de la matriz rala, vease abajo el renglón [0,1]
#tiene 3 valores asignados (0.166,0.217,0.556).

#El 1er parámetro señala indica que se seleccionará un valor
#de entre 0 y 19 para la 1er columna (que representa el índice
#de los renglones de la matriz rala que expresa); el 2do
#parámetro indica que se seleccionara al azar valores entre
#0 y 4 (que representa el índice de las columnas de la matriz
#rala que expresa); el 3er parámetro indica el número de elementos
#a generar 

#Fijaremos una semilla para reproducibilidad
np.random.seed(102)
MC=mf.generate_test_data(20, 5, 25)
print('Matriz generada con mf.generate_test_data(20,5,25):\n')
for i in range(MC.shape[0]):
    if MC[i,0]==0:
        print('\033[1;30;41m',MC[i,:])
    else:
        print('\033[0;0;0m',MC[i,:])
print('Tamaño MC:\n',MC.shape)

Matriz generada con mf.generate_test_data(20,5,25):

 [ 0.     1.     0.166]
 [ 19.      4.      0.306]
 [ 14.      3.      0.147]
 [ 18.      4.      0.223]
 [ 18.     1.     0.54]
 [ 15.      3.      0.749]
 [ 2.     2.     0.428]
 [ 11.      3.      0.105]
 [ 9.     3.     0.753]
 [ 8.     5.     0.598]
 [ 13.      0.      0.487]
 [ 8.     4.     0.975]
 [ 9.     2.     0.802]
 [ 15.      3.      0.333]
 [ 7.     5.     0.549]
 [ 4.     5.     0.312]
 [ 16.      4.      0.062]
 [ 6.     1.     0.903]
 [ 18.      0.      0.861]
 [ 3.    3.    0.63]
 [ 0.     1.     0.217]
 [ 0.     1.     0.556]
 [ 16.      5.      0.109]
 [ 6.     5.     0.551]
 [ 7.     0.     0.581]
Tamaño MC:
 (25, 3)


### Método mf.MF()
**CREAR OBJETO MF CON PARÁMETROS PARA EL MODELO DE FACTORIZACIÓN**

In [23]:
#Defínase un objeto de tipo MF para obtener la factorización deseada
mf_netflix=mf.MF()

#veamos algunas de las propiedades del objeto mf_netflix
print('Tipo de objeto: ',type(mf_netflix))
print('Instance variables: ',mf_netflix.__dict__)

Tipo de objeto:  <class 'libmf.mf.MF'>
Instance variables:  {'model': None, '_options': <libmf.mf.MFParam object at 0x7fb304095b70>, 'i': None, 'j': None}


In [24]:
#Hasta el momento no se han definido los parámetros para correr el
#modelo de factorización, sin embargo por default se asignan algunos
#valores que a continuación se muestran:
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=8
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


In [25]:
#Observa que entre los atributos están los parámetros para que corra libmf
#y que definen el modelo (son los últimos 13; los que no les antecede y finalizas
#con "__")
print('Atributos: ', dir(mf_netflix._options))

Atributos:  ['__class__', '__ctypes_from_outparam__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_b_base_', '_b_needsfree_', '_fields_', '_objects', 'copy_data', 'do_nmf', 'eta', 'fun', 'k', 'lambda_p1', 'lambda_p2', 'lambda_q1', 'lambda_q2', 'nr_bins', 'nr_iters', 'nr_threads', 'quiet']


In [27]:
#Sin embargo podemos definir parámetros distintos pasando los argumentos
#Aquí lo hacemos en forma de kwargs, es decir, con el nombre de las variables
mf_netflix=mf.MF(fun=0,k=10,nr_threads=12,nr_bins=26,nr_iters=20,lambda_p1=0.04,lambda_p2=0.0,lambda_q1=0.04,lambda_q2=0.0,eta=0.1,do_nmf=False,quiet=False,copy_data=True)
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=10
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


In [28]:
#Si optas por usar los parámetros por default excepto algunos
#puedes pasar los parámetros que quieres definir diferente
mf_netflix=mf.MF(eta=0.11,nr_iters=21, k=5)

#Nota que en este ejemplo como no definiste k=10, se inicializa con k=8
#pues es el valor por default
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=5
nr_threads=12
nr_bins=26
nr_iters=21
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1100
do_nmf= False
quiet= False
copy_data= True


**AJUSTE DEL MODELO DE FACTORIZACIÓN**

In [29]:
#Generamos una matriz rala al azar y la dividimos en entrenamiento y prueba
#para probar las funciones de factorización
np.random.seed(104)
random.seed(104)
MC=crear_matriz_rala_compacta(20,5,0.70)
print('El tamaño de MC es:',MC.shape)
MC_valida, MC_entrena=muestra_val_ent(pd.DataFrame(MC,columns=['usuario_id', 'peli_id', 'calif']),0.3)

MC_valida=MC_valida.to_numpy()
print('El tamaño de MC_valida es:',MC_valida.shape)
print('La matriz MC_valida queda así:\n',MC_valida)
MC_entrena=MC_entrena.to_numpy()
print('El tamaño de MC_entrena es:',MC_entrena.shape)

#IMPLEMENTAMOS MODELO CON MC_entrena
mf_netflix.fit(MC_entrena)

El tamaño de MC es: (70, 3)
El tamaño de MC_valida es: (7, 3)
La matriz MC_valida queda así:
 [[19  4  5]
 [16  4  5]
 [17  4  3]
 [15  4  1]
 [19  3  3]
 [16  3  1]
 [13  3  3]]
El tamaño de MC_entrena es: (63, 3)


**VALIDACIÓN CRUZADA**

In [30]:
#Para obtener la validación cruzada basta con llamar al método, pudiendo
#usar más de 5 folds (que son los que tiene por default)
print('Validación cruzada con 5 folds: ',mf_netflix.mf_cross_validation(MC_entrena))
print('Validación cruzada con 10 folds: ',mf_netflix.mf_cross_validation(MC_entrena,folds=10))

Validación cruzada con 5 folds:  5.605566760981551
Validación cruzada con 10 folds:  5.64388684302252


**OBTENCIÓN DE PREDICCIONES**

In [31]:
#Para hacer la predicción se le pasa sólo los
#índices de los elementos a predecir, es decir
#una base con sólo 2 columnas que indican el 
#numero de renglon y columnas
pred=mf_netflix.predict(MC_valida[:,0:2])
pred

array([ 5.198,  4.825,  8.489,  4.825,  3.287,  4.825,  4.825],
      dtype=float32)

In [32]:
print('Los valores verdaderos de MC_valida son:',MC_valida[:,2])

Los valores verdaderos de MC_valida son: [5 5 3 1 3 1 3]


In [33]:
print('Los resultados guardados en model están dispuestos como atributos: \n',dir(mf_netflix.model))

Los resultados guardados en model están dispuestos como atributos: 
 ['P', 'Q', '__class__', '__ctypes_from_outparam__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_b_base_', '_b_needsfree_', '_fields_', '_objects', 'b', 'fun', 'k', 'm', 'n']


In [34]:
imprime_atributos_model(mf_netflix.model)

Resultados model:
fun=0
m=20
n=20
k=20
b=4.8254
P=0.2455
Q=0.6670


**OBTENCIÓN MATRIZ Q**

In [35]:
Q=mf_netflix.q_factors()
print('Tamaño Q:',Q.shape)
print('Matriz Q:\n',Q)

Tamaño Q: (20, 5)
Matriz Q:
 [[ 0.667  1.343  1.086  1.3    1.157]
 [ 2.921  1.046  0.029  2.373  1.583]
 [ 1.272  0.352  0.301  2.018 -0.194]
 [ 0.101  0.74   0.969  0.996  0.843]
 [ 2.364  1.042  0.229  0.453  1.384]
 [ 1.355  3.005  2.902 -1.094  1.981]
 [   nan    nan    nan    nan    nan]
 [ 0.104  2.023  1.258  1.411  1.138]
 [   nan    nan    nan    nan    nan]
 [ 0.623  1.223  0.783  0.959  1.145]
 [ 0.929  0.475  0.655  2.293  0.828]
 [ 1.444  0.796  1.584  0.914  1.214]
 [ 1.812  1.259  1.189  0.823  1.61 ]
 [   nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan]
 [ 0.431  1.148  1.44   3.112  0.522]
 [ 0.724  0.594  1.621  1.907  2.427]
 [   nan    nan    nan    nan    nan]
 [ 0.827  1.668  0.587  1.622  0.729]
 [ 1.168  0.863  0.591  0.463  0.714]]


**OBTENCIÓN MATRIZ P**

In [36]:
P=mf_netflix.p_factors()
print('Tamaño P:',P.shape)
print('Matriz P:\n',P)

Tamaño P: (20, 5)
Matriz P:
 [[ 0.245  2.037  2.47  -1.25   1.073]
 [ 0.394  0.35   0.272  0.582  0.408]
 [-0.047  1.497  1.234  1.129  0.643]
 [ 1.493  1.077  0.438  0.61   1.068]
 [-0.418  0.959  0.854  0.62   0.081]
 [-0.338  0.528  1.138  2.796  0.406]
 [ 3.121  1.916  1.089  1.244  2.234]
 [ 0.053  0.759  0.76   0.188  0.777]
 [ 0.332  0.532 -0.027  0.218 -0.034]
 [ 2.313  1.78   1.216  1.053  1.915]
 [ 0.752  0.561  1.768  2.029  2.613]
 [ 0.847  0.893  1.04   1.123  1.13 ]
 [ 2.454  1.697  0.952  2.495  1.828]
 [   nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan]
 [   nan    nan    nan    nan    nan]
 [ 2.314  0.839  0.652  3.679  0.235]
 [   nan    nan    nan    nan    nan]
 [ 1.112  0.405  1.272  0.713  1.107]]


**OBTENCIÓN MATRIZ R**

In [37]:
R=P@np.transpose(Q)
print('Tamaño R:',R.shape)
print('Matriz R:\n',R)

Tamaño R: (20, 20)
Matriz R:
 [[ 5.199  1.652 -0.957  3.586  4.189 17.116    nan  6.711    nan  4.61
   0.835  6.048  6.646    nan    nan  2.671  5.613    nan  3.806  3.694]
 [ 2.256  3.552  1.802  1.485  2.188  2.547    nan  2.377    nan  1.912
   2.383  2.306  2.614    nan    nan  2.987  3.033    nan  2.311  1.484]
 [ 5.53   5.161  2.993  3.965  3.134  8.054    nan  6.9      nan  4.587
   4.597  4.892  5.231    nan    nan  7.325  6.57     nan  5.484  2.949]
 [ 4.945  8.639  3.434  2.879  6.507  7.98     nan  4.96     nan  4.398
   4.468  5.561  6.804    nan    nan  4.966  6.185    nan  5.056  3.978]
 [ 2.837  1.407  1.299  2.181  0.602  4.277    nan  3.938    nan  2.269
   2.116  2.179  2.107    nan    nan  4.124  3.032    nan  2.821  1.191]
 [ 5.823  6.876  5.662  4.585  1.844  2.176    nan  6.872    nan  4.473
   7.43   4.785  4.36     nan    nan 11.013  8.231    nan  6.102  2.319]
 [10.038 17.643  7.05   5.91  13.28  16.211    nan  9.868    nan  8.892
   9.225 11.607 13.985    nan

In [38]:
#Podemos ver cómo se ve la matriz R en forma compacta
compactar_matriz_rala(R)
pd.DataFrame(compactar_matriz_rala(R))

,0,1,2
0,0.0,0.0,5.198862
1,0.0,1.0,1.652449
2,0.0,2.0,-0.956831
3,0.0,3.0,3.586059
4,0.0,4.0,4.188980
...,...,...,...
220,19.0,12.0,6.406300
221,19.0,15.0,5.573576
222,19.0,16.0,7.153277
223,19.0,18.0,4.306216


In [39]:
#Notese que R justo contiene algunas de las estimaciones arrojadas por
#el método predict, pero no todas
comparativo=pd.DataFrame(columns=['ren','col','calif','R_calif_est','mf_netflix_predict'])
for i in range(MC_valida.shape[0]):
    comparativo.loc[i] = [MC_valida[i,0], MC_valida[i,1], MC_valida[i,2],R[MC_valida[i,0],MC_valida[i,1]],pred[i]]
display(comparativo)

#Observa que R no contiene todas las estimaciones que tiene predict
#checamos unos casos particulares: 4, 9, 11, 12
#nota que todos esos usuarios aparecen tanto en MC_entrena como 
#en MC_valida, y sin embargo la matriz R no tiene las calificaciones
#ni del usuario 13, 15 y 16
df_MC_valida=pd.DataFrame(MC_valida,columns=['ren','col','calif'])
df_MC_entrena=pd.DataFrame(MC_entrena,columns=['ren','col','calif'])
print('Usuarios 13, 15 y 16 en MC_valida:')
display(df_MC_valida[df_MC_valida.ren.isin([13,15,16])])
print('Usuarios 13, 15 y 16 en MC_entrena:')
display(df_MC_entrena[df_MC_entrena.ren.isin([13,15,16])])


#Obsérvese las películas que los usuarios 13, 15 y 16
#Tanto en entrena como en valida aparecen estos usuarios
#La base valida sólo tiene calificaciones de las películas
#3 y 4
print('Se calificaron películas 3 y 4 en MC_valida?...sí')
display(df_MC_valida[df_MC_valida.col.isin([3,4])])
print('Se calificaron películas 3 y 4 en MC_entrena?...sí')
display(df_MC_entrena[df_MC_entrena.col.isin([3,4])])


,ren,col,calif,R_calif_est,mf_netflix_predict
0,19.0,4.0,5.0,5.197598,5.197598
1,16.0,4.0,5.0,NaN,4.825397
2,17.0,4.0,3.0,8.488568,8.488568
3,15.0,4.0,1.0,NaN,4.825397
4,19.0,3.0,3.0,3.287389,3.287389
5,16.0,3.0,1.0,NaN,4.825397
6,13.0,3.0,3.0,NaN,4.825397


Usuarios 13, 15 y 16 en MC_valida:


,ren,col,calif
1,16,4,5
3,15,4,1
5,16,3,1
6,13,3,3


Usuarios 13, 15 y 16 en MC_entrena:


,ren,col,calif
32,15,2,1
34,16,1,5
40,15,0,3
56,13,2,2


Se calificaron películas 3 y 4 en MC_valida?...sí


,ren,col,calif
0,19,4,5
1,16,4,5
2,17,4,3
3,15,4,1
4,19,3,3
5,16,3,1
6,13,3,3


Se calificaron películas 3 y 4 en MC_entrena?...sí


,ren,col,calif
2,18,3,4
3,5,4,1
10,2,3,1
11,3,3,1
13,1,4,3
14,18,4,2
17,14,4,1
21,7,3,5
24,11,4,2
27,6,3,5


# Librería libmf

In [41]:
#ASEGURESE DE PONER LA UBICACIÓN CORRECTA DE LA UBICACIÓN DE LA CARPETA libmf
#Prueba correr el DEMO que viene en el la librería libmf

################################################################################################################
################## CORRÍ ESTO CONSIDERANDO QUE TUVE UNA COPIA DE LA CARPETA DE #################################
########################## LIBMF AL MISMO NIVEL DE Ejercicio_base_muestra_netflix ##############################
################################################################################################################
#%cd /home/miuser/libmf/demo
%cd /home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/demo/
!pwd
!bash demo.sh

/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/demo
/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/demo
g++ -Wall -O3 -pthread -std=c++0x -march=native -fopenmp -DUSESSE -DUSEOMP -c -fPIC -o mf.o mf.cpp
g++ -Wall -O3 -pthread -std=c++0x -march=native -fopenmp -DUSESSE -DUSEOMP -o mf-train mf-train.cpp mf.o
g++ -Wall -O3 -pthread -std=c++0x -march=native -fopenmp -DUSESSE -DUSEOMP -o mf-predict mf-predict.cpp mf.o
--------------------------------
Real-valued matrix factorization
--------------------------------
iter      tr_rmse      va_rmse          obj
   0       2.4804       1.3263   3.2505e+04
   1       1.2018       1.0784   8.9375e+03
   2       0.9106       1.0375   5.8782e+03
   3       0.8026       1.0178   4.9452e+03
   4       0.7464       1.0121   4.5159e+03
   5       0.6989       1.0068   4.1723e+03
   6       0.6536       1.0081   3.9015e+03
   7       0.6081       1.0097   3.6482e+03
   8       0.5596       1.0083   3.3892e+03
  

In [59]:
%cd /home/miuser/libmf/demo

/home/miuser/libmf/demo


In [42]:
#train a model using the default parameters
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9774   2.2897e+04
   1       1.0585   8.8300e+03
   2       0.8699   6.9933e+03
   3       0.8149   6.4923e+03
   4       0.7851   6.2068e+03
   5       0.7642   6.0061e+03
   6       0.7498   5.8820e+03
   7       0.7397   5.7511e+03
   8       0.7335   5.7207e+03
   9       0.7248   5.6410e+03
  10       0.7167   5.5583e+03
  11       0.7121   5.5264e+03
  12       0.7051   5.4380e+03
  13       0.6986   5.4035e+03
  14       0.6961   5.3763e+03
  15       0.6919   5.3477e+03
  16       0.6864   5.2997e+03
  17       0.6823   5.2656e+03
  18       0.6780   5.2287e+03
  19       0.6743   5.2182e+03


In [43]:
#train a model with the following regularization coefficients:
#coefficient of L1-norm regularization on P = 0.05
#coefficient of L1-norm regularization on Q = 0.05
#coefficient of L2-norm regularization on P = 0.01
#coefficient of L2-norm regularization on Q = 0.01
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train -l1 0.05 -l2 0.01 real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9594   2.2287e+04
   1       0.9935   8.0411e+03
   2       0.8678   6.8846e+03
   3       0.8016   6.3247e+03
   4       0.7704   6.0652e+03
   5       0.7464   5.8773e+03
   6       0.7262   5.7217e+03
   7       0.7093   5.5923e+03
   8       0.6946   5.4858e+03
   9       0.6781   5.3667e+03
  10       0.6646   5.2648e+03
  11       0.6521   5.1844e+03
  12       0.6388   5.1004e+03
  13       0.6244   4.9988e+03
  14       0.6113   4.9153e+03
  15       0.5999   4.8508e+03
  16       0.5869   4.7700e+03
  17       0.5738   4.6869e+03
  18       0.5619   4.6176e+03
  19       0.5507   4.5531e+03


In [44]:
#train a model with the following regularization coefficients:
#coefficient of L1-norm regularization on P = 0.05
#coefficient of L1-norm regularization on Q = 0
#coefficient of L2-norm regularization on P = 0.01
#coefficient of L2-norm regularization on Q = 0.03
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train -l1 0.015,0 -l2 0.01,0.005 real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.9508   1.9507e+04
   1       1.1678   7.3011e+03
   2       0.8453   4.0607e+03
   3       0.7961   3.6576e+03
   4       0.7611   3.3891e+03
   5       0.7385   3.2199e+03
   6       0.7226   3.1068e+03
   7       0.7070   2.9953e+03
   8       0.6923   2.8944e+03
   9       0.6787   2.8039e+03
  10       0.6653   2.7154e+03
  11       0.6505   2.6192e+03
  12       0.6382   2.5419e+03
  13       0.6245   2.4582e+03
  14       0.6109   2.3773e+03
  15       0.5968   2.2949e+03
  16       0.5835   2.2182e+03
  17       0.5700   2.1425e+03
  18       0.5570   2.0712e+03
  19       0.5445   2.0044e+03


In [45]:
#train a model with the following regularization coefficients:
#coefficient of L1-norm regularization on P = 0.05
#coefficient of L1-norm regularization on Q = 0
#coefficient of L2-norm regularization on P = 0.01
#coefficient of L2-norm regularization on Q = 0.03
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train -l1 0.015,0 -l2 0.01,0.005 real_matrix.tr.txt model

iter      tr_rmse          obj
   0       1.8967   1.8463e+04
   1       1.0987   6.5169e+03
   2       0.8494   4.0957e+03
   3       0.7914   3.6210e+03
   4       0.7631   3.4026e+03
   5       0.7361   3.2037e+03
   6       0.7214   3.0978e+03
   7       0.7049   2.9815e+03
   8       0.6921   2.8924e+03
   9       0.6783   2.8025e+03
  10       0.6656   2.7170e+03
  11       0.6520   2.6305e+03
  12       0.6386   2.5459e+03
  13       0.6249   2.4610e+03
  14       0.6111   2.3787e+03
  15       0.5976   2.2987e+03
  16       0.5844   2.2234e+03
  17       0.5708   2.1472e+03
  18       0.5576   2.0749e+03
  19       0.5449   2.0069e+03


In [46]:
#train a BMF model using logarithmic loss and the following parameters:
#coefficient of L1-norm regularization on P = 0
#coefficient of L1-norm regularization on Q = 0.01
#latent factors = 100
#iterations = 30
#learning rate = 0.02
#threads = 4
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train -f 5 -l1 0,0.02 -k 100 -t 30 -r 0.02 -s 4 binary_matrix.tr.txt model

iter   tr_logloss          obj
   0       0.6994   4.1037e+03
   1       0.6949   4.0124e+03
   2       0.6915   3.9377e+03
   3       0.6892   3.8798e+03
   4       0.6872   3.8309e+03
   5       0.6859   3.7930e+03
   6       0.6848   3.7615e+03
   7       0.6840   3.7348e+03
   8       0.6833   3.7122e+03
   9       0.6828   3.6929e+03
  10       0.6824   3.6763e+03
  11       0.6821   3.6622e+03
  12       0.6817   3.6488e+03
  13       0.6815   3.6376e+03
  14       0.6813   3.6277e+03
  15       0.6811   3.6188e+03
  16       0.6810   3.6108e+03
  17       0.6808   3.6032e+03
  18       0.6807   3.5965e+03
  19       0.6806   3.5904e+03
  20       0.6805   3.5848e+03
  21       0.6804   3.5796e+03
  22       0.6804   3.5747e+03
  23       0.6803   3.5704e+03
  24       0.6803   3.5661e+03
  25       0.6802   3.5623e+03
  26       0.6802   3.5588e+03
  27       0.6802   3.5554e+03
  28       0.6801   3.5523e+03
  29       0.6801   3.5492e+03


In [47]:
#use real_matrix.te.txt for hold-out validation
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train -p real_matrix.te.txt real_matrix.tr.txt model

iter      tr_rmse      va_rmse          obj
   0       1.9774       1.1544   2.2897e+04
   1       1.1294       1.0517   9.6420e+03
   2       0.8681       1.0276   6.9599e+03
   3       0.8163       1.0200   6.5054e+03
   4       0.7831       1.0086   6.1943e+03
   5       0.7677       1.0095   6.0253e+03
   6       0.7518       1.0060   5.8868e+03
   7       0.7396       1.0049   5.7795e+03
   8       0.7325       1.0040   5.6994e+03
   9       0.7244       1.0043   5.6249e+03
  10       0.7183       1.0045   5.5695e+03
  11       0.7118       1.0056   5.5211e+03
  12       0.7065       1.0049   5.4715e+03
  13       0.7010       1.0060   5.4176e+03
  14       0.6957       1.0037   5.3651e+03
  15       0.6920       1.0058   5.3352e+03
  16       0.6873       1.0065   5.3111e+03
  17       0.6814       1.0089   5.2599e+03
  18       0.6777       1.0093   5.2244e+03
  19       0.6743       1.0072   5.2122e+03


In [48]:
#do five fold cross validation
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train -v 5 real_matrix.tr.txt

fold      rmse
   0    1.4021
   1    1.4231
   2    1.5066
   3    1.5455
   4    1.5097
 avg    1.4774


In [49]:
#do non-negative matrix factorization with generalized KL-divergence
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train -f 2 --nmf real_matrix.tr.txt

iter       tr_gkl          obj
   0       0.9952   8.0187e+03
   1       0.2734   4.1972e+03
   2       0.1777   3.6801e+03
   3       0.1569   3.5439e+03
   4       0.1455   3.4395e+03
   5       0.1372   3.3901e+03
   6       0.1319   3.3433e+03
   7       0.1273   3.2944e+03
   8       0.1238   3.2557e+03
   9       0.1213   3.2456e+03
  10       0.1182   3.2196e+03
  11       0.1165   3.1980e+03
  12       0.1141   3.1746e+03
  13       0.1135   3.1746e+03
  14       0.1107   3.1355e+03
  15       0.1111   3.1538e+03
  16       0.1088   3.1253e+03
  17       0.1085   3.1339e+03
  18       0.1070   3.1178e+03
  19       0.1060   3.1052e+03


In [50]:
#do not print message to screen
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train --quiet real_matrix.tr.txt

In [51]:
#do disk-level training
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-train --disk real_matrix.tr.txt

iter      tr_rmse          obj
   0       1.7131   1.8232e+04
   1       1.1925   1.0366e+04
   2       0.8651   6.9268e+03
   3       0.8060   6.3841e+03
   4       0.7787   6.1476e+03
   5       0.7582   5.9554e+03
   6       0.7406   5.8034e+03
   7       0.7288   5.6923e+03
   8       0.7217   5.6269e+03
   9       0.7134   5.5558e+03
  10       0.7113   5.5109e+03
  11       0.7045   5.4513e+03
  12       0.7005   5.4372e+03
  13       0.6918   5.3614e+03
  14       0.6865   5.3223e+03
  15       0.6814   5.2615e+03
  16       0.6808   5.2644e+03
  17       0.6777   5.2280e+03
  18       0.6699   5.1742e+03
  19       0.6664   5.1561e+03


In [52]:
#do prediction
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-predict real_matrix.te.txt model output

RMSE = 1.0072


In [53]:
#do prediction and output MAE
!/home/miuser/proyecto-final-equipo5-mno-2020-1/Implementation/libmf/mf-predict -e 1 real_matrix.te.txt model output

MAE = 0.7846


# Obtención de muestra base netflix

In [54]:
#ASEGURESE DE PONER LA UBICACIÓN CORRECTA DE LA UBICACIÓN DE LA CARPETA DE LAS BASES
#movies_titles_fix.csv y dat_muestra_nflix.csv

################################################################################################################
################## LA ÚNICA BASE DE FELIPE QUE ENCONTRÉ EN EL REPO ESTÁ AQUÍ ###################################
####### ESTA FUE LA ÚNICA PARTE DE LA OBTENCIÓN DE MUESTRA BASE NETFLIX QUE VOLVÍ A CORRER #####################
################################################################################################################
%cd /home/miuser/proyecto-final-equipo5-mno-2020-1/Sampling_Design/Reference_Files/
#Se guarda una variable con una muestra de las base de netflix ("movies_title_fix.csv")
#A continuación se presentan las características de las películas, con el objeto
#de conocer un poco más los datos utilizados (pero esta no será la base usada para el modelo)
pelis_nombres=pd.read_csv('movies_title_fix.csv',names=["peli_id", "año", "nombre"])
pelis_nombres.head(10) #para ver cómo se encuentran los datos

/home/miuser/proyecto-final-equipo5-mno-2020-1/Sampling_Design/Reference_Files


,peli_id,año,nombre
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
6,7,1992.0,8 Man
7,8,2004.0,What the #$*! Do We Know!?
8,9,1991.0,Class of Nuke 'Em High 2
9,10,2001.0,Fighter


In [50]:
#La base tiene peli_id, usuario_id_orig, calif, fecha y usuario_id. Se trabajará solo
#con las columnas de usuario, pelicula y calificación

################################################################################################################
################## A PARTIR DE AQUÍ SE REQUIERE LA BASE DE FELIPE dat_muestra_nflix.csv ########################
#################################### PERO NO LA UBICQUÉ EN EL REPO #############################################
################################################################################################################
%cd /home/miuser/proyecto-final-equipo5-mno-2020-1/Sampling_Design/
df_muestra_netflix=pd.read_csv('dat_muestra_nflix.csv')
df_muestra_netflix_libmf=df_muestra_netflix[['usuario_id', 'peli_id', 'calif']]
df_muestra_netflix_libmf.shape

(20968941, 3)

In [51]:
#Se genera una base de validación que contendrá el 20% de los datos
#y una base de entrenamiento que tendrá el restante 80%
random.seed(28882)
data_valida, data_entrena=muestra_val_ent(df_muestra_netflix_libmf,0.2)

################################################################################################################
#################### ESTO GUARDA LAS BASES QUE USA ESTE NOTEBOOK EN LA CARPETA DONDE ########################
################################# ESTA GUARDADO ESTE NOTEBOOK ###############################################
################################################################################################################
#puedes guardar las bases como un .csv dentro del directorio donde está este notebook
data_valida.to_csv(r'data_valida.csv', index = False)    
data_entrena.to_csv(r'data_entrena.csv', index = False)    

#Base de todos los usuarios
usuarios=pd.DataFrame(df_muestra_netflix_libmf['usuario_id'].unique(), columns=['usuario_id'])
#Muestreo de usuarios
valida_usuarios=usuarios.sample(frac=0.2)  

#Base de todas las películas
peliculas=pd.DataFrame(df_muestra_netflix_libmf['peli_id'].unique(), columns=['peli_id'])
#Muestreo de películas
valida_pelis=peliculas.sample(frac=0.2)

In [52]:
#df_valida tiene las columnas usuario_id, peli_id y calif, pero
#sólo incluye los usuarios muestreados en valida_usuarios
df_valida=pd.merge(df_muestra_netflix_libmf,valida_usuarios, on ='usuario_id', how='inner')
#data_valida filtra de la base df_valida los pelis_id muestreas
#en valida_pelis
data_valida=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
#guardamos data_valida como un .csv dentro del directorio donde está este notebook
data_valida.to_csv(r'muestra_valida.csv', index = False)
left= pd.merge(df_muestra_netflix_libmf, data_valida, how='left',on=['usuario_id', 'peli_id'])
data_entrena=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]

In [53]:
display(data_valida.head(5))
data_valida.shape

,usuario_id,peli_id,calif
0,4,128,4
1,89,128,5
2,125,128,2
3,452,128,1
4,642,128,2


(877113, 3)

In [54]:
display(data_entrena.head(5))
data_entrena.shape

,usuario_id,peli_id,calif_x
0,1,1,3
1,2,1,3
2,3,1,4
3,4,1,4
4,5,1,4


(20091828, 3)

In [55]:
#podemos cambiar el directorio para que se guarde la base
#dentro del directorio donde están las funciones de libmf
#se guarda con la extensión .tr.txt porque es la extensión
#que requiere la paquetería de libmf

################################################################################################################
################# CON ESTE CÓDIGO GUARDA LAS BASES CREADAS DE data_valida.te.txt Y #############################
################### data_entrena.tr.txt REQUERIDAS PARA CORRER UN MODELO EN LIBMF ##############################
############ PERO IGUAL ESTO QUEDA DESECHADO SI SÓLO SE VA A OCUPAR LA LIBRERÍA LIBMF-PYTHON ###################
################################################################################################################
%cd /home/miuser/libmf/demo
np.savetxt('data_valida.te.txt', data_valida.to_numpy(), delimiter=' ', fmt="%.0f")
np.savetxt('data_entrena.tr.txt', data_entrena.to_numpy(), delimiter=' ', fmt="%.0f")

/home/miuser/libmf/demo


In [56]:
#La base de df_muestra_valida_libmf ha quedado dividida por
#dos dataframes exluyenntes: data_valida y data_entrena
print("La suma de elementos de data_valida + data_entrena:",data_valida.shape[0] + data_entrena.shape[0])
print("El total de elementos de df_muestra_netflix_libmf es:",df_muestra_netflix_libmf.shape[0])

La suma de elementos de data_valida + data_entrena: 20968941
El total de elementos de df_muestra_netflix_libmf es: 20968941


In [57]:
modelo_base_ref(data_valida.to_numpy())

,row,col,mean_score_by_row,mean_score_by_col,total_score_mean
row,,,,,
4,4,128,4.207792,2.954545,3.609339
4,4,187,4.207792,3.078089,3.609339
4,4,571,4.207792,3.953481,3.609339
4,4,586,4.207792,3.247706,3.609339
4,4,708,4.207792,3.371808,3.609339
...,...,...,...,...,...
99912,99912,17307,5.000000,4.521127,3.609339
99928,99928,16169,2.000000,3.131050,3.609339
99937,99937,17293,3.000000,2.980172,3.609339


# Implementación modelo

In [ ]:
# EMPEZAR A CONTRUIR UN MODELO "PEQUEÑO", QUE LUEGO SEA ESCALABLE A LA BASE GRANDE EN LA INSTANCIA
# PUEDE HACERSE EN OTRO ARCHIVO, YA INCLUSO CON LAS FUNCIONES AQUÍ EXPUESTAS EN UN .PY (LAS QUE SE USEN)

In [ ]:
################################################################################################################
###################### ESTO CORRE UN MODELO SIMPLE CON LIBMF A PARTIR DE LA BASE ###############################
############################### CREADAS ARRIBA, PERO ESTO YA NO SE CORRIÓ ######################################
################################################################################################################

#Pruebese los diferentes modelos ejemplo con data_entrena y data_valida
#En general estos algoritmos corren muy rápido
!/home/miuser/libmf/mf-train data_entrena.tr.txt model